# Notes

This assignment is devoted to `pandas`. It covers indexing and filtering, and some `groupby` and `join` operations. The assignment roughly corresponds to Week 4 and the beginning of Week 5 of the course.

The main dataset you'll be using is [Titanic](https://www.kaggle.com/c/titanic).

In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import numpy as np
import pandas as pd

In [4]:
STUDENT = "Shaul Solomon"
#Partnered with Adam Cohn
ASSIGNMENT = 4
TEST = False

In [6]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 16

ModuleNotFoundError: No module named 'solutions'

# Indexing and filtering

### 1. Fixing age (1 point).

There are several known mistakes in the Titanic dataset.

Namely, [Julia Florence Siegel](https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html) (Mrs. Tyrell William Cavendish) is mistakenly marked as being 76 years old (the age she actually died, but many years after Titanic).

You must replace age value for her with her actual age at the time (25) and return the dataset. Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return a copy of the dataframe, and not perform replacement in original dataframe. Structure and indexing must be the same as in input.

In [7]:
#We know her Passenger ID is 988
def fix_age(df):
    updated_df = df.copy()
    updated_df.loc[988,"Age"] = 25
    return updated_df

In [8]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

NameError: name 'total_grade' is not defined

### 2. Embarkment port distribution (1 point).

You must find the value counts for embarkment port (`Embarked` column) for the passengers, who travelled in 3-d class, were male and between 20 and 30 years old (both inclusive). No need to treat missing values separately.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. You must return series, indexed with values from `Embarked`, according to `.value_counts()` method semantics.

In [9]:
def embarked_stats(df):
    """Calculate embarkment port statistics."""
    #First filter out those who travelled in 3-d class, were male and between 20 and 30 years old 
    embarked = df.loc[(df.Age.between(20,30,inclusive=True)) & (df['Pclass'] == 3) & (df['Sex'] == 'male')].Embarked.value_counts()
    return embarked

# print(embarked_stats(df))
    

In [10]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, embarked_stats)

[Shaul Solomon] [Problem 2] correct!


### 3. Fill missing age values (1 point).

Some age values are missing in the Titanic dataset. You need to calculate average age, and fill missing age values in `Age` column.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure, but without missing values in `Age` column.

In [11]:
def fix_age(df):
    """Fix missing age values."""
    mydf = df.copy()
    mydf['Age'] = mydf.Age.fillna(mydf.Age.mean())
    return mydf

In [12]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age)

[Shaul Solomon] [Problem 3] correct!


### 4. Child travelling alone (1 point).

You must find a child (`Age<10`) on-board, who was travelling without siblings or parents and find a name of her nursemaid.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two strings, collected from `Name` column, with one being child's name and second being nursemaid's name. It's known, that there's only one child like this.

In [13]:
def get_nursemaid(df):
    child =  df[(df["Age"] < 10) & (df["SibSp"] == 0) & (df["Parch"] == 0)]["Ticket"]
    names = df.loc[(df["Ticket"] == child.iloc[0])].sort_values(by='Age', ascending=True)['Name']
    return tuple(names)

# print(get_nursemaid(df))

In [14]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_nursemaid)

[Shaul Solomon] [Problem 4] incorrect: 
correct = ('Emanuel, Miss. Virginia Ethel', 'Dowdell, Miss. Elizabeth')
submitted = ('Dowdell, Miss. Elizabeth', 'Emanuel, Miss. Virginia Ethel')



### 5. Port with the most children embarked (1 point).

You must find, which port had the largest percentage of children (`Age<10`) embarked, i.e. number of children divided by total number of passengers embarked.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a single string with port letter.

In [15]:
def get_port(df):
    return (df[df["Age"]<10].groupby("Embarked").size()/df.groupby("Embarked").size()).idxmax()

In [16]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_port)

[Shaul Solomon] [Problem 5] correct!


### 6. Passengers per ticket (2 points).

Calculate average and maximum number of passengers per ticket.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a tuple of two values.

In [17]:
def get_ticket_stats(df):
    return tuple(df.groupby('Ticket').size().aggregate(['mean','max']))

In [18]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_ticket_stats)

[Shaul Solomon] [Problem 6] correct!


### 7. Fare per passenger (3 points).

For each individual ticket, you must calculate fare per person for that ticket, and then calculate averages for each class. Note, that you will need to apply `groupby` and then you may consider using `.first()` of resulting `DataFrameGroupBy`.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements, indexed by class.

In [19]:
def get_fare_per_pass(df):
    """Calculate fare per passenger for different classes."""
    ### If we are trying to get the average cost of the tickets per person per class
    
    #ticketgroup = df.groupby(['Ticket','Pclass'])
    #avg_prices = ticketgroup.first()['Fare'] / ticketgroup.size()
    #return avg_prices.groupby('Pclass').mean()
    
    ### If we are trying to get the average cost of the fare per person per class
    df2 = df.copy()
    df2['fare_pp'] = df2['Fare'] / df2.groupby('Ticket').Ticket.transform('count')
    return df2.groupby('Pclass')['fare_pp'].mean()

    

# get_fare_per_pass(df)

In [20]:
PROBLEM_ID = 7

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, get_fare_per_pass)

[Shaul Solomon] [Problem 7] correct!


### 8. Fill missing age values (3 points).

In problem 3 you filled missing age values with average for all passengers. Now, you need to fill them according to class and sex. For example, for a female passenger from 2d class, missing age value must be filled with average age of females in 2d class.

In this problem, you may need joins and `.apply()`, although there are several ways to get the same result.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a **new** dataframe with the same structure as input, but without missing values in `Age` column.

In [10]:
def fix_age_groupped(df):
    """Fill missing age values."""
    data = df.copy()
    meanAge = data.groupby(['Pclass','Sex'])['Age'].mean()
    data = data.reset_index().set_index(['Pclass','Sex'])
    data['Age'] = data['Age'].fillna(meanAge)
    data = data.reset_index()
    data.index = df.index
    return data[df.columns]

In [22]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, fix_age_groupped)

[Shaul Solomon] [Problem 8] failed: 
exception is "None of ['PassengerId'] are in the columns"



### 9. Finding couples (3 points).

Based on the code from Lecture 5, build a dataframe of couples. Filter it by survival status: select those couples, in which only one of spouses survived or none of two. Built survival statistics by class, i.e. ratio of couples with partial survival divided by total number of couples in class.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be `Series` with three elements indexed by values from `Pclass` column.

In [23]:
def find_couples(df):
    family_names = (df
                .replace(re.compile(r'\s+\(.*\)'), '')
                .replace(re.compile("Mrs."), "Mr."))[["Name", "Sex","Survived","Pclass"]]
    womens_names = family_names[(family_names.Sex=="female") & family_names.Name.str.contains("Mr.")]
    mens_names = family_names[(family_names.Sex=="male") & family_names.Name.str.contains("Mr.")]
    family = pd.merge(mens_names,womens_names, how="inner", left_on="Name", right_on="Name", suffixes=("","_f"))[["Pclass","Survived","Survived_f"]]
    family.dropna(inplace=True)
    return family[(family['Survived'] != 1.0) | (family['Survived_f'] != 1.0)].groupby("Pclass").size() / family.groupby("Pclass").size()

In [24]:
PROBLEM_ID = 8

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_couples)

[Shaul Solomon] [Problem 9] correct!


In [25]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")

Shaul Solomon: 75
